In [1]:
import QpixAsicArray as qparray
from QpixAsic import PixelHit, ProcQueue
import random
import numpy as np
random.seed(2)

In [2]:
qpa = qparray.QpixAsicArray(3, 3, debug=0.0, deltaT=2e-6)
qpa._daqNode._debugLevel = 0
qpa.Calibrate(0.2)
# qpa.Calibrate(0.2)
print(qpa._daqNode.daqHits)

performing calibration..
calibration complete in 100043 steps!
current time is 0.2000000000001587
9


In [3]:
# qpa.Calibrate(0.2)
qpa.Calibrate(1.0)

performing calibration..
calibration complete in 500047 steps!
current time is 1.200001999992832


In [4]:
guessFrq = {}
for k, v in qpa._daqNode.pixelData.items():
    asic = k
    # print(asic)
    daqTicks = []
    asicTicks = []
    for daqTick, asicTick in v:
        daqTicks.append(daqTick)
        asicTicks.append(asicTick)
    print("asic ticks:", asicTicks)
    guessFrq[asic] = (asicTicks[1] - asicTicks[0]) / (daqTicks[1] - daqTicks[0])  * qpa._daqNode.fOsc
for asic in qpa:
    key = f"({asic.row},{asic.col})"
    print(f"{key} percent diff: {(guessFrq[key] - asic.fOsc) / asic.fOsc * 100:1.2E}")

asic ticks: [265, 27923532, 55846799]
asic ticks: [493, 48344326]
asic ticks: [484, 46496140]
asic ticks: [795, 52089708]
asic ticks: [761, 47734956]
asic ticks: [746, 47315111]
asic ticks: [995, 47892381]
asic ticks: [1069, 51057120]
asic ticks: [1365, 52979135]
(0,0) percent diff: 4.50E-07
(0,1) percent diff: 1.89E-06
(0,2) percent diff: -3.97E-07
(1,0) percent diff: 1.24E-06
(1,1) percent diff: 1.24E-06
(1,2) percent diff: 2.47E-06
(2,0) percent diff: 1.01E-07
(2,1) percent diff: -2.82E-08
(2,2) percent diff: -8.10E-07
